In [2]:
import argparse
import os
import torch
import numpy as np
from exp.exp_IL_DiffTSF import Exp_IL_DiffTSF

parser = argparse.ArgumentParser(description='IL_DiffTSF')

parser.add_argument('--model', type=str, default='IL_DiffTSF')
value="ETTh1" #ETTh1,ETTm2,electrans,weather,exchange
parser.add_argument('--data', type=str, default=value, help='data')
parser.add_argument('--root_path', type=str, default=os.path.abspath('../')+'/datasets', help='root path of the data file')
parser.add_argument('--data_path', type=str, default=value+'.csv', help='data file')    #electrans.csv
parser.add_argument('--features', type=str, default='M', help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h', help='15min or 1H or 1D')
parser.add_argument('--checkpoints', type=str, default= 'model_parameters/'+value, help='location of model checkpoints')
parser.add_argument('--label_len', type=int, default=int(24*7) , help='start token length of Informer decoder')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')

parser.add_argument('--d_model', type=int, default=256, help='dimension of model')
parser.add_argument('--num_workers', type=int, default=8, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=20, help='train epochs')
parser.add_argument('--batch_size', type=int, default=1, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=1e-4, help='optimizer learning rate')
parser.add_argument('--lradj', type=str, default='type1', help='none, type1, type2')
parser.add_argument('--inverse', action='store_true', help='inverse output data', default=False)
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3',help='device ids of multile gpus')
parser.add_argument('--n_times', type=int, default=100, help='diffusion steps')
parser.add_argument('--sampling', type=bool, default=False, help='sampling of outcomes')
parser.add_argument('--sampling_times', type=int, default=30, help='number of outcomes')
parser.add_argument('--train_est', type=bool, default=False , help='train estimator')
parser.add_argument('--offset', type=bool, default=False, help='offset of the output')

args = parser.parse_args("")
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]
    
torch.cuda.empty_cache()


import random

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)  # Replace 42 with your desired seed


In [ ]:

Exp = Exp_IL_DiffTSF
def runtrain(args):
        mae_list=[]
        crps_list=[]
        mse_list=[]
        index =args.model +"_"+str(args.label_len)+"_"+str(args.pred_len)
        resultsMAE,resultsCRPS,resultsMSE=[],[],[]
        mae_list=[]
        crps_list,mse_list=[],[]
        for ii in range(args.itr):
               
                # setting record of experiments
                setting = '{}_d{}_ft{}_ll{}_pl{}_dm{}_n_times_{}_ii_{}_OF_{}'.format(
                        index, args.data, args.features,args.label_len, args.pred_len,
                        args.d_model,args.n_times,args.reg,args.offset)
                
                exp = Exp(args) # set experiments
                print('>>>>>>>start training {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)
                torch.cuda.empty_cache()

                resultout,crpsret=exp.test(setting,True)

                mae,mse,rmse,mspe,shape =resultout

                print(setting)
                mse_list.append(mse)
                mse_np=np.array(mse_list)
                mae_list.append(mae)
                mae_np=np.array(mae_list)
                crps_list.append(np.mean(crpsret))
                crps_np=np.array(crps_list)
        
                print("predlen:"+str(args.pred_len)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.round(np.mean(crps_np),3))+"±"+str(np.round(np.std(crps_np),4)))
                print("predlen:"+str(args.pred_len)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.round(np.mean(mse_np),3))+"±"+str(np.round(np.std(mse_np),4)))
                print("predlen:"+str(args.pred_len)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.round(np.mean(mae_np),3))+"±"+str(np.round(np.std(mae_np),4)))
                resultsCRPS.append(str(ii)+"_"+"predlen:"+str(args.pred_len)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.round(np.mean(crps_np),3))+"±"+str(np.round(np.std(crps_np),4)))
                resultsMSE.append(str(ii)+"_"+"predlen:"+str(args.pred_len)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.round(np.mean(mse_np),3))+"±"+str(np.round(np.std(mse_np),4)))
                resultsMAE.append(str(ii)+"_"+"predlen:"+str(args.pred_len)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.round(np.mean(mae_np),3))+"±"+str(np.round(np.std(mae_np),4)))
                f= open(os.path.abspath('')+"/results/"+str(args.data)+"/"+str(args.pred_len)+"_d_ff"+str(args.d_model)+"_ntimes"+str(args.n_times)+"_"+str(args.data)+"_"+str(args.reg)+"_f"+str(args.features)+"_condk"+str(args.condk)+".txt","w+")
                f.write(str(setting)+"\n"+str(resultsCRPS) +"\n"+str(resultsMSE)+"\n"+str(resultsMAE))
                f.close()

        torch.cuda.empty_cache()
        return str(np.round(np.mean(mae_np),3))

dataname='ETTh1'
args.data =dataname
args.checkpoints='model_parameters/'+dataname

data_parser = {
'exchange':{'data':'exchange_all.csv','T':'OT','M':[8,8,8],'S':[1,1,1],'MS':[8,8,1],'freqin':'1D'},
'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'1H'},
'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'1H'},
'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'1H'},
'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'15min'},
'weather':{'data':'weather.csv','T':'OT','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1],'freqin':'1H'},
'electrans':{'data':'electrans.csv','T':'OT','M':[32,32,32],'S':[1,1,1],'MS':[321,321,1],'freqin':'1H'},
}
if args.data in data_parser.keys():
        data_info = data_parser[args.data]
        args.data_path = data_info['data']
        args.target = data_info['T']
        args.enc_in, args.dec_in, args.c_out = data_info[args.features]
        args.freq=data_info['freqin']

args.detail_freq = args.freq
print('Args in experiment:')
print(args)

for i in range(0,5):
        torch.cuda.empty_cache()
        args.d_model=256
        args.reg=100
        args.n_times=100
        items= [[168,24],[168,96],[336,168],[720,336]] #for ETTh1
        args.batch_size=32
        args.label_len =items[i][0]
        args.pred_len =items[i][1]
        print(args.label_len)
        print(args.pred_len)
        runtrain(args)



Args in experiment:
Namespace(model='IL_DiffTSF', data='ETTh1', root_path='/home/uservan/Desktop/TS_Diff/datasets', data_path='ETTh1.csv', features='M', target='OT', freq='1H', checkpoints='model_parameters/ETTh1', label_len=168, pred_len=24, d_model=256, num_workers=8, itr=1, train_epochs=20, batch_size=32, patience=3, learning_rate=0.0001, lradj='type1', inverse=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', n_times=100, sampling=False, sampling_times=30, train_est=False, offset=False, enc_in=7, dec_in=7, c_out=7, detail_freq='1H', lambdareg=100)
168
24
Use GPU: cuda:0
Trainable Parameters:2690137
Trainable Parameters:286727
>>>>>>>start training IL_DiffTSF_168_24_dETTh1_ftM_ll168_pl24_dm256_n_times_100_ii_100_OF_False>>>>>>>>>>>>>>>>>>>>>>>>>>
train 12003
val 1551
No File, Train new
No EST File, Train new


  0%|          | 0/376 [00:00<?, ?it/s]